# 목표
-  상가, 전력, 주거, 복지시설 관련 데이터를 종합하여여 시,군,구 별로 정리하여 하나의 데이터로 종합한다.




 

In [11]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

# 데이터프레임 출력 옵션 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

print("라이브러리 로드 및 기본 설정이 완료되었습니다.")

라이브러리 로드 및 기본 설정이 완료되었습니다.


In [12]:
# 1. 주거 실거래 데이터 로드 및 확인
data_path = '../data/'
house_file = '주거실거래정보.csv'

try:
    house_df = pd.read_csv(data_path + house_file, encoding='utf-8')
    print("주거 실거래 데이터 로드 성공.")
    print("\n데이터 정보:")
    house_df.info()
    print("\n데이터 샘플:")
    display(house_df.head())
except FileNotFoundError:
    print(f"오류: 파일을 찾을 수 없습니다. ({data_path}{house_file})")
    print("데이터 파일의 위치를 확인해 주세요. '데이터병합.ipynb'가 있는 'code' 디렉토리의 상위 'data' 디렉토리에 파일이 있어야 합니다.")
except Exception as e:
    print(f"데이터 로드 중 오류 발생: {e}")

주거 실거래 데이터 로드 성공.

데이터 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1538030 entries, 0 to 1538029
Data columns (total 22 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   NO            1538030 non-null  int64  
 1   시군구           1538030 non-null  object 
 2   번지            1538022 non-null  object 
 3   본번            1538030 non-null  int64  
 4   부번            1538030 non-null  int64  
 5   단지명           1276469 non-null  object 
 6   전월세구분         1538030 non-null  object 
 7   전용면적(㎡)       1538030 non-null  float64
 8   계약년월          1538030 non-null  int64  
 9   계약일           1538030 non-null  int64  
 10  보증금(만원)       1538030 non-null  object 
 11  월세금(만원)       1538030 non-null  object 
 12  층             1538030 non-null  int64  
 13  건축년도          1538030 non-null  int64  
 14  도로명           1538030 non-null  object 
 15  계약기간          1538030 non-null  object 
 16  계약구분          1538030 non-null  object 
 17  

,NO,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세금(만원),층,건축년도,도로명,계약기간,계약구분,갱신요구권 사용,종전계약 보증금(만원),종전계약 월세(만원),주택유형,건물명
0,1,서울특별시 성동구 마장동,819,819,0,마장동중앙하이츠,전세,84.800,202301,31,"49,200",0,1,1999,마장로 262,202301~202404,갱신,-,"46,200",0.0,아파트,NaN
1,2,서울특별시 성동구 마장동,782-1,782,1,홀스케슬,전세,28.420,202301,31,"10,000",0,2,2011,마조로18길 25,202302~202502,갱신,-,"10,000",0.0,아파트,NaN
2,3,경기도 용인시 처인구 남사읍 아곡리,681,681,0,e편한세상용인한숲시티5단지,월세,84.826,202301,31,"2,000",100,21,2018,한숲로 83,202302~202502,신규,-,NaN,NaN,아파트,NaN
3,4,전북특별자치도 전주시 완산구 태평동,305,305,0,태평아이파크,전세,84.794,202301,31,"30,000",0,5,2022,태진로 35,202302~202502,신규,-,NaN,NaN,아파트,NaN
4,5,경기도 수원시 장안구 정자동,313-1,313,1,동신2단지,전세,59.660,202301,31,"16,000",0,4,1988,장안로 211,202302~202502,신규,-,NaN,NaN,아파트,NaN


In [13]:
# 2. 주거 데이터 전처리
house_processed = house_df.copy()

# '전월세구분'이 '월세'인 데이터만 필터링
house_processed = house_processed[house_processed['전월세구분'] == '월세'].reset_index(drop=True)

# '월세금(만원)'과 '보증금(만원)' 컬럼을 숫자형으로 변환
# 콤마(,)를 제거하고 숫자로 변환하며, 오류 발생 시 NaN으로 처리
for col in ['월세금(만원)', '보증금(만원)']:
    house_processed[col] = house_processed[col].astype(str).str.replace(',', '', regex=False)
    house_processed[col] = pd.to_numeric(house_processed[col], errors='coerce')

# 월세금이 0이거나 없는(NaN) 데이터는 분석 의미가 없으므로 제거
house_processed.dropna(subset=['월세금(만원)'], inplace=True)
house_processed = house_processed[house_processed['월세금(만원)'] > 0].reset_index(drop=True)

# '시군구' 컬럼에서 분석 단위인 '시도-시군구' 정보 추출하는 함수 정의
def extract_sigungu(address):
    """
    주소 문자열에서 '시도 시군구' 또는 '특별시/광역시 구' 부분을 추출한다.
    예: '경기도 용인시 처인구 남사읍' -> '경기도 용인시 처인구'
        '서울특별시 성동구 마장동' -> '서울특별시 성동구'
        '세종특별자치시 한솔동' -> '세종특별자치시' (세종시는 시군구 레벨이 없음)
    """
    parts = address.split()
    if not parts:
        return None
    
    # 세종특별자치시는 단일 레벨
    if parts[0] == '세종특별자치시':
        return parts[0]
    
    # 행정시(구 존재)의 경우: ex) 경기도 수원시 장안구
    if len(parts) >= 3 and (parts[1].endswith('시') or parts[1].endswith('군')) and parts[2].endswith('구'):
        return f"{parts[0]} {parts[1]} {parts[2]}"
    
    # 일반 시/군/구의 경우: ex) 서울특별시 성동구, 경기도 이천시
    if len(parts) >= 2:
        return f"{parts[0]} {parts[1]}"
        
    return parts[0]

# '시군구_병합' 컬럼 생성
house_processed['시군구_병합'] = house_processed['시군구'].apply(extract_sigungu)

# '강원도'를 '강원특별자치도'로 통일 (데이터 정합성 확보)
house_processed['시군구_병합'] = house_processed['시군구_병합'].str.replace('강원도', '강원특별자치도', regex=False)

# 전처리 결과 확인
print("주거 데이터 전처리 완료.")
print(f"월세 데이터 개수: {len(house_processed)}")
print("\n'시군구_병합' 컬럼 고유값 일부:")
print(list(house_processed['시군구_병합'].unique()[:15]))
print("\n전처리된 데이터 샘플:")
display(house_processed[['시군구', '시군구_병합', '월세금(만원)', '보증금(만원)']].head())

주거 데이터 전처리 완료.
월세 데이터 개수: 689941

'시군구_병합' 컬럼 고유값 일부:
['경기도 용인시 처인구', '서울특별시 광진구', '경상북도 포항시 북구', '경기도 수원시 영통구', '경기도 수원시 장안구', '충청남도 천안시 동남구', '경기도 수원시 권선구', '경상남도 창원시 진해구', '경기도 성남시 분당구', '서울특별시 용산구', '서울특별시 동대문구', '전북특별자치도 전주시 완산구', '전북특별자치도 전주시 덕진구', '경상남도 창원시 마산합포구', '경기도 고양시 덕양구']

전처리된 데이터 샘플:


,시군구,시군구_병합,월세금(만원),보증금(만원)
0,경기도 용인시 처인구 남사읍 아곡리,경기도 용인시 처인구,100,2000
1,서울특별시 광진구 중곡동,서울특별시 광진구,160,5000
2,경상북도 포항시 북구 장성동,경상북도 포항시 북구,55,10000
3,경기도 수원시 영통구 영통동,경기도 수원시 영통구,74,3000
4,경기도 수원시 장안구 정자동,경기도 수원시 장안구,100,5000


In [15]:
# 3. 주거비 부담 지수 계산을 위한 소득 데이터 로드 및 전처리 (수정 2)
income_file = '시군구별_종합소득세.csv'

try:
    # 소득 데이터 로드 시, 소득 금액 컬럼을 문자열 타입으로 강제하여 로드
    try:
        income_df = pd.read_csv(
            data_path + income_file,
            header=1,
            encoding='utf-8',
            dtype={'종합소득금액 (백만원)': str}  # 이 부분을 추가하여 오류를 해결
        )
    except Exception:
        income_df = pd.read_csv(
            data_path + income_file,
            header=1,
            encoding='cp949',
            dtype={'종합소득금액 (백만원)': str}
        )

    print("소득 데이터 로드 성공.")

    # 컬럼명 변경
    income_df.rename(columns={
        '시군구별(1)': '시도',
        '시군구별(2)': '시군구',
        '종합소득금액 (백만원)': '지역별_소득수준'
    }, inplace=True)

    # 불필요한 행 제거
    income_df = income_df[income_df['시도'] != '지역별']
    income_df = income_df[(income_df['시군구'] != '소계') | (income_df['시도'] == '세종')].reset_index(drop=True)

    # 지역명 형식 통일
    sido_mapping = {
        '서울': '서울특별시', '부산': '부산광역시', '대구': '대구광역시',
        '인천': '인천광역시', '광주': '광주광역시', '대전': '대전광역시',
        '울산': '울산광역시', '세종': '세종특별자치시', '경기': '경기도',
        '강원': '강원특별자치도', '충북': '충청북도', '충남': '충청남도',
        '전북': '전북특별자치도', '전남': '전라남도', '경북': '경상북도',
        '경남': '경상남도', '제주': '제주특별자치도'
    }
    income_df['시도_정식명칭'] = income_df['시도'].map(sido_mapping)

    def create_merge_key(row):
        if row['시도'] == '세종':
            return row['시도_정식명칭']
        else:
            return f"{row['시도_정식명칭']} {row['시군구']}"
    income_df['시군구_병합'] = income_df.apply(create_merge_key, axis=1)

    # 숫자형으로 변환
    income_df['지역별_소득수준'] = pd.to_numeric(income_df['지역별_소득수준'].str.replace(',', ''), errors='coerce')

    # 최종 데이터 선택
    income_df = income_df[['시군구_병합', '지역별_소득수준']].dropna()

    print("\n소득 데이터 전처리 완료.")
    print("\n전처리된 소득 데이터 샘플:")
    display(income_df.head())

except FileNotFoundError:
    print(f"오류: 파일을 찾을 수 없습니다. ({data_path}{income_file})")
    print(f"'{data_path}' 디렉토리에 '{income_file}' 파일이 있는지 확인해주세요.")
except Exception as e:
    print(f"데이터 로드 또는 처리 중 오류 발생: {e}")

소득 데이터 로드 성공.

소득 데이터 전처리 완료.

전처리된 소득 데이터 샘플:


,시군구_병합,지역별_소득수준
0,서울특별시 강남구,20309561
1,서울특별시 강동구,3787186
2,서울특별시 강북구,1408596
3,서울특별시 강서구,4094519
4,서울특별시 관악구,3099592


In [17]:
# 4. 주거 관련 지표 계산 (지역명 표준화 및 계산 방식 수정 반영)
import re

print("주거 데이터 지표 계산을 시작합니다 (지역명 표준화 및 부담 지수 계산 방식 수정 포함).")

# --- 4-1. 지표 계산을 위한 사전 계산 ---
# (이전과 동일)
house_processed['건물_노후도'] = 2023 - house_processed['건축년도']
house_processed = house_processed[house_processed['건물_노후도'] >= 0].copy()
house_processed = house_processed[house_processed['전용면적(㎡)'] > 0].copy()
house_processed['평당_가격'] = house_processed['월세금(만원)'] / (house_processed['전용면적(㎡)'] * 0.3025)

def get_region_type(sigungu_name):
    if sigungu_name.startswith(('서울특별시', '인천광역시', '경기도')): return '수도권'
    elif sigungu_name.startswith(('부산광역시', '대구광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시')): return '광역시'
    else: return '도'

region_area_map = {'수도권': 34.2, '광역시': 35.6, '도': 39.0}
house_processed['지역구분'] = house_processed['시군구_병합'].apply(get_region_type)
house_processed['1인당_주거면적'] = house_processed['지역구분'].map(region_area_map)
house_processed['단위면적당_월세'] = house_processed['월세금(만원)'] / house_processed['전용면적(㎡)']
house_processed['1인당_표준화월세'] = house_processed['단위면적당_월세'] * house_processed['1인당_주거면적']
print("사전 계산 완료: 건물 노후도, 평당 가격, 1인당 표준화 월세")

# --- 4-2. 지역명 표준화 (행정구 통합) ---
special_cities = ['고양시', '부천시', '성남시', '수원시', '안산시', '안양시', '용인시',
                  '창원시', '포항시', '전주시', '천안시', '청주시']
def standardize_region_name(name):
    name = re.sub(r'\(.*\)', '', str(name)).strip()
    parts = name.split()
    if len(parts) > 2 and parts[1] in special_cities:
        return f"{parts[0]} {parts[1]}"
    return name

house_processed['시군구_병합_수정'] = house_processed['시군구_병합'].apply(standardize_region_name)
income_df['시군구_병합_수정'] = income_df['시군구_병합'].apply(standardize_region_name)
income_standardized = income_df[['시군구_병합_수정', '지역별_소득수준']].drop_duplicates(subset=['시군구_병합_수정'])
print("지역명 표준화 완료.")

# --- 4-3. 표준화된 키로 데이터 집계 및 병합 ---
print("표준화된 지역명을 기준으로 집계 및 병합 시작...")
house_agg = house_processed.groupby('시군구_병합_수정').agg(
    월세_평균_표준화=('1인당_표준화월세', 'mean'),
    월세_표준편차_표준화=('1인당_표준화월세', 'std'),
    평균_평당_가격=('평당_가격', 'mean'),
    건물_노후도_평균=('건물_노후도', 'mean'),
    거래량=('시군구_병합_수정', 'size')
).reset_index()
house_agg['월세_표준편차_표준화'].fillna(0, inplace=True)

house_final = pd.merge(house_agg, income_standardized, on='시군구_병합_수정', how='left')
house_final.rename(columns={'시군구_병합_수정': '시군구_병합'}, inplace=True)
print("집계 및 병합 완료.")

# --- 4-4. 최종 지표 계산 (수정된 주거비 부담 지수 적용) ---
house_final.dropna(subset=['지역별_소득수준'], inplace=True)

# 1. 주거비 부담 지수 (월 단위로 통일하여 계산)
#   - 연간 소득(백만원)을 월평균 소득(만원)으로 변환: (소득 / 12) * 100
house_final['월평균_소득수준_만원'] = (house_final['지역별_소득수준'] / 12) * 100
#   - (월세(만원) / 월소득(만원)) * 100
house_final['주거비_부담_지수'] = (house_final['월세_평균_표준화'] / house_final['월평균_소득수준_만원']) * 100

# 2. 기타 지수 계산
house_final['주택_시장_활력_지수'] = house_final['거래량'] * (1 + house_final['월세_표준편차_표준화'])
house_final['신규_주택_공급_지수'] = 1 / house_final['건물_노후도_평균'].replace(0, 1e-6)

# 불필요한 중간 계산 컬럼 제거
house_final.drop(columns=['월평균_소득수준_만원'], inplace=True)

print("모든 지표 계산 완료.")

# --- 최종 결과 확인 ---
print("\n최종 생성된 주거 데이터(house_final) 정보:")
house_final.info()
print("\n최종 주거 데이터 샘플:")
display(house_final.head())

주거 데이터 지표 계산을 시작합니다 (지역명 표준화 및 부담 지수 계산 방식 수정 포함).
사전 계산 완료: 건물 노후도, 평당 가격, 1인당 표준화 월세
지역명 표준화 완료.
표준화된 지역명을 기준으로 집계 및 병합 시작...
집계 및 병합 완료.
모든 지표 계산 완료.

최종 생성된 주거 데이터(house_final) 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   시군구_병합       228 non-null    object 
 1   월세_평균_표준화    228 non-null    float64
 2   월세_표준편차_표준화  228 non-null    float64
 3   평균_평당_가격     228 non-null    float64
 4   건물_노후도_평균    228 non-null    float64
 5   거래량          228 non-null    int64  
 6   지역별_소득수준     228 non-null    int64  
 7   주거비_부담_지수    228 non-null    float64
 8   주택_시장_활력_지수  228 non-null    float64
 9   신규_주택_공급_지수  228 non-null    float64
dtypes: float64(7), int64(2), object(1)
memory usage: 17.9+ KB

최종 주거 데이터 샘플:


,시군구_병합,월세_평균_표준화,월세_표준편차_표준화,평균_평당_가격,건물_노후도_평균,거래량,지역별_소득수준,주거비_부담_지수,주택_시장_활력_지수,신규_주택_공급_지수
0,강원특별자치도 강릉시,33.651822,19.986456,2.852454,20.943791,2295,1160203,0.000348,48163.916951,0.047747
1,강원특별자치도 고성군,37.536465,15.465865,3.181730,11.154762,84,117857,0.003822,1383.132628,0.089648
2,강원특별자치도 동해시,21.369696,13.275026,1.811375,19.161987,926,406705,0.000631,13218.674171,0.052187
3,강원특별자치도 삼척시,34.362456,26.415448,2.912690,22.138298,282,248577,0.001659,7731.156358,0.045171
4,강원특별자치도 속초시,42.095656,20.636705,3.568184,18.571325,1381,496214,0.001018,29880.289212,0.053846


In [19]:
# 6. 상가 데이터 재로드 및 전처리 (분기 수정 및 지역명 추출)

# --- 6-1. 분기 정보를 1, 2, 3, 4로 수정하여 데이터 재로드 ---
store_files = {
    '03': '상가(상권)정보_03.csv', '06': '상가(상권)정보_06.csv',
    '09': '상가(상권)정보_09.csv', '12': '상가(상권)정보_12.csv'
}
quarter_map = {'03': 1, '06': 2, '09': 3, '12': 4} # 분기 매핑
all_stores_df = []

print("분기 정보를 수정하여 상가 데이터를 다시 로드합니다...")
try:
    for month, file_name in store_files.items():
        cols_to_use = ['도로명주소', '상권업종대분류명', '상권업종중분류명', '상권업종소분류명', '표준산업분류명']
        temp_df = pd.read_csv(data_path + file_name, usecols=cols_to_use, encoding='utf-8')
        temp_df['분기'] = quarter_map[month] # 1, 2, 3, 4로 매핑
        all_stores_df.append(temp_df)
    
    store_df = pd.concat(all_stores_df, ignore_index=True)
    print("데이터 재로드 및 통합 완료.")

except Exception as e:
    print(f"오류 발생: {e}")

# --- 6-2. 주소 정보 전처리 및 시군구 단위 추출 ---
print("\n주소 정보 전처리를 시작합니다...")
# 도로명주소가 없는 결측치 제거
store_df.dropna(subset=['도로명주소'], inplace=True)

# 도로명주소에서 '시군구' 추출하는 함수 정의 (주거 데이터와 유사)
def extract_sigungu_from_store(address):
    try:
        parts = str(address).split()
        if not parts: return None
        if parts[0] == '세종특별자치시': return parts[0]
        # '경기도 수원시 장안구' 같은 형태 처리
        if len(parts) >= 3 and (parts[1].endswith('시') or parts[1].endswith('군')) and parts[2].endswith('구'):
            return f"{parts[0]} {parts[1]} {parts[2]}"
        if len(parts) >= 2: return f"{parts[0]} {parts[1]}"
        return parts[0]
    except (TypeError, IndexError):
        return None

# '시군구_병합' 컬럼 생성
store_df['시군구_병합'] = store_df['도로명주소'].apply(extract_sigungu_from_store)

# '강원도' -> '강원특별자치도' 통일
store_df['시군구_병합'] = store_df['시군구_병합'].str.replace('강원도', '강원특별자치도', regex=False)
store_df.dropna(subset=['시군구_병합'], inplace=True)

# 행정구 통합을 위한 표준화 키 생성 (이전에 정의한 함수 재사용)
store_df['시군구_병합_수정'] = store_df['시군구_병합'].apply(standardize_region_name)

print("전처리 완료.")
print("\n전처리된 상가 데이터 정보:")
store_df.info()
print("\n전처리된 상가 데이터 샘플:")
display(store_df[['도로명주소', '시군구_병합', '시군구_병합_수정', '분기']].head())

분기 정보를 수정하여 상가 데이터를 다시 로드합니다...
데이터 재로드 및 통합 완료.

주소 정보 전처리를 시작합니다...
전처리 완료.

전처리된 상가 데이터 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9709068 entries, 0 to 9709067
Data columns (total 8 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   상권업종대분류명   object
 1   상권업종중분류명   object
 2   상권업종소분류명   object
 3   표준산업분류명    object
 4   도로명주소      object
 5   분기         int64 
 6   시군구_병합     object
 7   시군구_병합_수정  object
dtypes: int64(1), object(7)
memory usage: 592.6+ MB

전처리된 상가 데이터 샘플:


,도로명주소,시군구_병합,시군구_병합_수정,분기
0,강원도 동해시 송정로 11,강원특별자치도 동해시,강원특별자치도 동해시,1
1,강원도 춘천시 동내면 외솔길19번길 80-36,강원특별자치도 춘천시,강원특별자치도 춘천시,1
2,강원도 정선군 남면 민둥산로 175-13,강원특별자치도 정선군,강원특별자치도 정선군,1
3,강원도 춘천시 스포츠타운길 458,강원특별자치도 춘천시,강원특별자치도 춘천시,1
4,강원도 원주시 지정면 신지정로 211,강원특별자치도 원주시,강원특별자치도 원주시,1


In [20]:
# 7. 상가 관련 지표 계산 및 집계

print("시군구별 상가 관련 지표 계산을 시작합니다...")

# --- 7-1. 분기별 상가 수 계산 (변화율 계산용) ---
quarterly_counts = store_df.groupby(['시군구_병합_수정', '분기']).size().unstack(fill_value=0)

# 1분기 또는 4분기 데이터가 없을 수 있으므로 컬럼 확인 후 추가
for q in [1, 2, 3, 4]:
    if q not in quarterly_counts.columns:
        quarterly_counts[q] = 0

# 상가 변화율 계산: (4분기 수 - 1분기 수) / 1분기 수
# 1분기 수가 0일 경우 변화율은 0으로 처리 (신규 상권)
change_rate = (quarterly_counts[4] - quarterly_counts[1]) / quarterly_counts[1]
change_rate.replace([np.inf, -np.inf], 0, inplace=True) # 1분기 0개 -> 4분기 N개일 때 inf 방지
change_rate.fillna(0, inplace=True)
change_rate = change_rate.reset_index(name='상가_변화율')
print("분기별 상가 변화율 계산 완료.")

# --- 7-2. 다양성 지수 계산 함수 정의 ---
def shannon_index(series):
    proportions = series.value_counts(normalize=True)
    return -np.sum(proportions * np.log(proportions))

def hhi_index(series):
    proportions = series.value_counts(normalize=True)
    return np.sum((proportions * 100) ** 2)

# --- 7-3. 시군구별 주요 지표 집계 ---
print("주요 지표(평균 상가 수, 다양성 지수)를 집계합니다...")
# '표준산업분류명' 결측치 제거 후 집계
store_df_filtered = store_df.dropna(subset=['표준산업분류명'])

store_agg = store_df_filtered.groupby('시군구_병합_수정').agg(
    다양성_지수_새넌=('표준산업분류명', shannon_index),
    다양성_지수_HHI=('표준산업분류명', hhi_index)
).reset_index()

# 연간 평균 상가 수 추가
store_agg['상가_개수_평균'] = store_df.groupby('시군구_병합_수정').size().reset_index(name='count')['count'] / 4

# --- 7-4. 업종별 상가 수 집계 ---
print("업종별 상가 수를 집계합니다...")
category_counts = store_df.pivot_table(
    index='시군구_병합_수정',
    columns='상권업종대분류명',
    values='도로명주소',
    aggfunc='count',
    fill_value=0
)
category_counts.columns = ['업종별상가수_' + col for col in category_counts.columns]
category_counts.reset_index(inplace=True)

# --- 7-5. 모든 지표 병합 ---
print("계산된 모든 지표를 하나로 병합합니다...")
store_agg = pd.merge(store_agg, change_rate, on='시군구_병합_수정', how='left')
store_agg = pd.merge(store_agg, category_counts, on='시군구_병합_수정', how='left')
store_agg.fillna(0, inplace=True) # 병합 과정에서 생긴 NaN을 0으로 처리

print("상가 데이터 집계 완료.")
print("\n최종 생성된 상가 데이터(store_agg) 정보:")
store_agg.info()
print("\n최종 상가 데이터 샘플:")
display(store_agg.head())

시군구별 상가 관련 지표 계산을 시작합니다...
분기별 상가 변화율 계산 완료.
주요 지표(평균 상가 수, 다양성 지수)를 집계합니다...
업종별 상가 수를 집계합니다...
계산된 모든 지표를 하나로 병합합니다...
상가 데이터 집계 완료.

최종 생성된 상가 데이터(store_agg) 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   시군구_병합_수정       230 non-null    object 
 1   다양성_지수_새넌       230 non-null    float64
 2   다양성_지수_HHI      230 non-null    float64
 3   상가_개수_평균        230 non-null    float64
 4   상가_변화율          230 non-null    float64
 5   업종별상가수_과학·기술    230 non-null    int64  
 6   업종별상가수_교육       230 non-null    int64  
 7   업종별상가수_보건의료     230 non-null    int64  
 8   업종별상가수_부동산      230 non-null    int64  
 9   업종별상가수_소매       230 non-null    int64  
 10  업종별상가수_수리·개인    230 non-null    int64  
 11  업종별상가수_숙박       230 non-null    int64  
 12  업종별상가수_시설관리·임대  230 non-null    int64  
 13  업종별상가수_예술·스포츠   230 non-null    int64  
 14  업종별상가

,시군구_병합_수정,다양성_지수_새넌,다양성_지수_HHI,상가_개수_평균,상가_변화율,업종별상가수_과학·기술,업종별상가수_교육,업종별상가수_보건의료,업종별상가수_부동산,업종별상가수_소매,업종별상가수_수리·개인,업종별상가수_숙박,업종별상가수_시설관리·임대,업종별상가수_예술·스포츠,업종별상가수_음식
0,강원특별자치도 강릉시,4.412261,276.430534,14603.50,0.031213,3172,2020,304,1351,12950,6784,4893,2040,2401,22499
1,강원특별자치도 고성군,3.992554,435.338219,2287.00,0.072836,344,212,22,193,1951,578,1978,283,392,3195
2,강원특별자치도 동해시,4.432885,284.248181,5303.50,0.029242,895,656,117,221,5706,2716,859,737,835,8472
3,강원특별자치도 삼척시,4.210664,368.206013,3866.25,0.069641,601,296,46,167,3722,1582,1480,581,578,6412
4,강원특별자치도 속초시,4.325667,319.685911,6360.25,0.024616,1170,562,155,691,6360,2495,1372,702,1065,10869


In [27]:
# 8. 상가 관련 추가 지표 계산 (수정된 코드 3)

print("상가 데이터에 추가 지표(업종 특화도, 인구/면적 대비 상가 수)를 계산합니다.")

population_data = None
area_data = None

# --- 8-1. 추가 데이터 로드 및 전처리 (파일 구조 반영) ---
try:
    print("인구 및 면적 데이터를 로드합니다...")
    # 인구 데이터 로드
    welfare_df = pd.read_csv(data_path + '사회복지시설수.csv', encoding='utf-8', header=2)
    welfare_df.columns = ['시도', '시군구', '인구수'] + welfare_df.columns[3:].tolist()
    welfare_df = welfare_df[['시도', '시군구', '인구수']]
    welfare_df = welfare_df[welfare_df['시도'] != '총계']
    welfare_df = welfare_df[(welfare_df['시군구'] != '소계') | (welfare_df['시도'] == '세종특별자치시')].reset_index(drop=True)
    welfare_df['시군구_병합'] = welfare_df.apply(lambda r: r['시도'] if r['시도'] == '세종특별자치시' else r['시도'] + ' ' + r['시군구'], axis=1)
    welfare_df['시군구_병합_수정'] = welfare_df['시군구_병합'].apply(standardize_region_name)
    population_data = welfare_df[['시군구_병합_수정', '인구수']].drop_duplicates('시군구_병합_수정')
    population_data['인구수'] = pd.to_numeric(population_data['인구수'], errors='coerce')
    print("'사회복지시설수.csv' 처리 완료.")

    # 면적 데이터 로드 (파일 구조에 맞게 수정)
    area_df = pd.read_csv(data_path + '시군구별면적.csv', encoding='utf-8', skiprows=1)
    area_df = area_df.iloc[:, [0, -1]] # 첫 열(지역명)과 마지막 열(최신 면적)만 사용
    area_df.columns = ['지역명', '면적(㎢)']
    area_df.dropna(subset=['지역명', '면적(㎢)'], inplace=True)
    area_df = area_df[area_df['지역명'] != '전체']

    def convert_area_region_name(name):
        sido_short_map = {
            '서울': '서울특별시', '부산': '부산광역시', '대구': '대구광역시',
            '인천': '인천광역시', '광주': '광주광역시', '대전': '대전광역시',
            '울산': '울산광역시', '세종': '세종특별자치시', '경기': '경기도',
            '강원': '강원특별자치도', '충북': '충청북도', '충남': '충청남도',
            '전북': '전북특별자치도', '전남': '전라남도', '경북': '경상북도', '경남': '경상남도', '제주': '제주특별자치도'
        }
        parts = name.split()
        if not parts: return None
        if parts[0] in sido_short_map:
            return f"{sido_short_map[parts[0]]} {' '.join(parts[1:])}".strip()
        return name if name in sido_short_map.values() else None

    area_df['시군구_병합'] = area_df['지역명'].apply(convert_area_region_name)
    area_df.dropna(subset=['시군구_병합'], inplace=True)
    area_df['시군구_병합_수정'] = area_df['시군구_병합'].apply(standardize_region_name)
    area_data = area_df[['시군구_병합_수정', '면적(㎢)']].drop_duplicates('시군구_병합_수정')
    area_data['면적(㎢)'] = pd.to_numeric(area_data['면적(㎢)'], errors='coerce')
    print("'시군구별면적.csv' 처리 완료.")

except FileNotFoundError as e:
    print(f"\n[오류] 파일을 찾을 수 없습니다: {e.filename}")
except Exception as e:
    print(f"\n[오류] 추가 데이터 로드 또는 처리 중 오류 발생: {e}")

# --- 8-2. 업종 특화도 (LQ) 계산 ---
print("\n업종 특화도를 계산합니다...")
region_total_stores = category_counts.set_index('시군구_병합_수정').sum(axis=1)
nation_industry_counts = category_counts.drop(columns='시군구_병합_수정').sum()
nation_total_stores = nation_industry_counts.sum()
lq_matrix = category_counts.set_index('시군구_병합_수정').copy()
for industry in lq_matrix.columns:
    if region_total_stores.empty or industry not in nation_industry_counts or nation_industry_counts[industry] == 0: continue
    lq_matrix[industry] = (lq_matrix[industry] / region_total_stores) / (nation_industry_counts[industry] / nation_total_stores)
specialization_score = (lq_matrix > 1.25).sum(axis=1).reset_index(name='업종_특화도')
print("업종 특화도 계산 완료.")

# --- 8-3. 모든 데이터 병합 ---
print("\n모든 상가 관련 데이터를 병합합니다...")
store_final = pd.merge(store_agg, specialization_score, on='시군구_병합_수정', how='left')

if population_data is not None and area_data is not None:
    store_final = pd.merge(store_final, population_data, on='시군구_병합_수정', how='left')
    store_final = pd.merge(store_final, area_data, on='시군구_병합_수정', how='left')
    
    # --- 8-4. 인구/면적 대비 지표 계산 ---
    store_final['인구_대비_상가_수'] = (store_final['상가_개수_평균'] / store_final['인구수']).replace([np.inf, -np.inf], 0).fillna(0) * 1000
    store_final['면적_대비_상가_수'] = (store_final['상가_개수_평균'] / store_final['면적(㎢)']).replace([np.inf, -np.inf], 0).fillna(0)
    print("인구/면적 대비 지표 계산 완료.")
else:
    print("[경고] 인구/면적 데이터가 로드되지 않아 추가 지표를 계산할 수 없습니다.")

store_final.rename(columns={'시군구_병합_수정': '시군구_병합'}, inplace=True)

# --- 최종 결과 확인 ---
print("\n추가 지표 계산 및 병합 완료.")
print("\n최종 생성된 상가 데이터(store_final) 정보:")
store_final.info()
print("\n최종 상가 데이터 샘플:")
display(store_final.head())

상가 데이터에 추가 지표(업종 특화도, 인구/면적 대비 상가 수)를 계산합니다.
인구 및 면적 데이터를 로드합니다...
'사회복지시설수.csv' 처리 완료.
'시군구별면적.csv' 처리 완료.

업종 특화도를 계산합니다...
업종 특화도 계산 완료.

모든 상가 관련 데이터를 병합합니다...
인구/면적 대비 지표 계산 완료.

추가 지표 계산 및 병합 완료.

최종 생성된 상가 데이터(store_final) 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   시군구_병합          230 non-null    object 
 1   다양성_지수_새넌       230 non-null    float64
 2   다양성_지수_HHI      230 non-null    float64
 3   상가_개수_평균        230 non-null    float64
 4   상가_변화율          230 non-null    float64
 5   업종별상가수_과학·기술    230 non-null    int64  
 6   업종별상가수_교육       230 non-null    int64  
 7   업종별상가수_보건의료     230 non-null    int64  
 8   업종별상가수_부동산      230 non-null    int64  
 9   업종별상가수_소매       230 non-null    int64  
 10  업종별상가수_수리·개인    230 non-null    int64  
 11  업종별상가수_숙박       230 non-null    int64  
 12  업종별상가수_시설관리·임대  230 non-nu

,시군구_병합,다양성_지수_새넌,다양성_지수_HHI,상가_개수_평균,상가_변화율,업종별상가수_과학·기술,업종별상가수_교육,업종별상가수_보건의료,업종별상가수_부동산,업종별상가수_소매,업종별상가수_수리·개인,업종별상가수_숙박,업종별상가수_시설관리·임대,업종별상가수_예술·스포츠,업종별상가수_음식,업종_특화도,인구수,면적(㎢),인구_대비_상가_수,면적_대비_상가_수
0,강원특별자치도 강릉시,4.412261,276.430534,14603.50,0.031213,3172,2020,304,1351,12950,6784,4893,2040,2401,22499,1,215603.0,1041.0,67.733288,14.028338
1,강원특별자치도 고성군,3.992554,435.338219,2287.00,0.072836,344,212,22,193,1951,578,1978,283,392,3195,1,27869.0,664.0,82.062507,3.444277
2,강원특별자치도 동해시,4.432885,284.248181,5303.50,0.029242,895,656,117,221,5706,2716,859,737,835,8472,1,91492.0,180.0,57.966817,29.463889
3,강원특별자치도 삼척시,4.210664,368.206013,3866.25,0.069641,601,296,46,167,3722,1582,1480,581,578,6412,2,65939.0,1188.0,58.633737,3.254419
4,강원특별자치도 속초시,4.325667,319.685911,6360.25,0.024616,1170,562,155,691,6360,2495,1372,702,1065,10869,2,83674.0,106.0,76.012262,60.002358


In [28]:
# 9. 전력 사용 정보 데이터 로드 및 기본 전처리

print("전력 사용 정보 데이터를 로드하고 기본 전처리를 수행합니다...")

try:
    elec_file = '전력사용정보.csv'
    # 사용자가 모든 파일 인코딩이 utf-8이라고 명시했으므로 utf-8 사용
    elec_df = pd.read_csv(data_path + elec_file, encoding='utf-8')
    print("전력 데이터 로드 성공.")

    # '시군구' 병합 키 생성
    # '시도', '시군구' 컬럼이 없는 경우를 대비한 예외 처리
    if '시도' in elec_df.columns and '시군구' in elec_df.columns:
        elec_df['시군구_병합'] = elec_df['시도'] + ' ' + elec_df['시군구']
    else:
        # 다른 이름의 컬럼이 있을 경우를 가정 (예: '지역')
        # 이 부분은 실제 파일 구조에 따라 조정될 수 있음
        raise KeyError("'시도' 또는 '시군구' 컬럼을 찾을 수 없습니다.")

    # 지역명 통일
    elec_df['시군구_병합'] = elec_df['시군구_병합'].str.replace('강원도', '강원특별자치도', regex=False)
    elec_df['시군구_병합_수정'] = elec_df['시군구_병합'].apply(standardize_region_name)

    print("기본 전처리 완료.")
    print("\n전처리된 전력 데이터 정보:")
    elec_df.info()
    print("\n전처리된 전력 데이터 샘플:")
    display(elec_df.head())

except FileNotFoundError:
    print(f"오류: 파일을 찾을 수 없습니다. ({data_path}{elec_file})")
except Exception as e:
    print(f"데이터 로드 또는 처리 중 오류 발생: {e}")

전력 사용 정보 데이터를 로드하고 기본 전처리를 수행합니다...
전력 데이터 로드 성공.
기본 전처리 완료.

전처리된 전력 데이터 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3002 entries, 0 to 3001
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   년월                  3002 non-null   int64 
 1   시도                  3002 non-null   object
 2   시군구                 3002 non-null   object
 3   대상가구수(호)            3002 non-null   object
 4   가구당 평균 전력 사용량(kWh)  3002 non-null   int64 
 5   가구당 평균 전기요금(원)      3002 non-null   object
 6   시군구_병합              3002 non-null   object
 7   시군구_병합_수정           3002 non-null   object
dtypes: int64(2), object(6)
memory usage: 187.8+ KB

전처리된 전력 데이터 샘플:


,년월,시도,시군구,대상가구수(호),가구당 평균 전력 사용량(kWh),가구당 평균 전기요금(원),시군구_병합,시군구_병합_수정
0,202307,강원도,강릉시,"131,388",210,"28,372",강원특별자치도 강릉시,강원특별자치도 강릉시
1,202307,강원도,고성군,"19,487",165,"25,718",강원특별자치도 고성군,강원특별자치도 고성군
2,202307,강원도,동해시,"53,935",211,"27,328",강원특별자치도 동해시,강원특별자치도 동해시
3,202307,강원도,삼척시,"44,647",168,"24,322",강원특별자치도 삼척시,강원특별자치도 삼척시
4,202307,강원도,속초시,"50,307",233,"30,190",강원특별자치도 속초시,강원특별자치도 속초시


In [29]:
# 10. 전력 사용량 지표 계산 및 집계

print("시군구별 전력 사용량 관련 지표 계산을 시작합니다...")

# --- 10-1. 데이터 타입 변환 및 월 정보 추출 ---
# '가구당 평균 전기요금(원)' 컬럼을 숫자형으로 변환
elec_df['가구당 평균 전기요금(원)'] = elec_df['가구당 평균 전기요금(원)'].str.replace(',', '').astype(int)
# '년월'에서 '월' 정보 추출
elec_df['월'] = elec_df['년월'] % 100
print("데이터 타입 변환 및 월 정보 추출 완료.")

# --- 10-2. 기본 지표 집계 ---
print("기본 지표(평균 사용량, 변동성, 효율)를 집계합니다...")
elec_agg = elec_df.groupby('시군구_병합_수정').agg(
    가구당_평균_전력사용량=('가구당 평균 전력 사용량(kWh)', 'mean'),
    가구당_평균_전기요금=('가구당 평균 전기요금(원)', 'mean'),
    전력사용량_std=('가구당 평균 전력 사용량(kWh)', 'std')
).reset_index()

# 전력소비 변동성 지수 (표준편차 / 평균)
elec_agg['전력소비_변동성_지수'] = elec_agg['전력사용량_std'] / elec_agg['가구당_평균_전력사용량']

# 전력 에너지 효율 (평균 요금 / 평균 사용량)
elec_agg['전력_에너지_효율'] = elec_agg['가구당_평균_전기요금'] / elec_agg['가구당_평균_전력사용량']

# 중간 계산에 사용된 컬럼 제거
elec_agg.drop(columns=['가구당_평균_전기요금', '전력사용량_std'], inplace=True)

# --- 10-3. 계절성 지수 계산 ---
print("전력소비 계절성 지수를 계산합니다...")
def get_season(month):
    if month in [6, 7, 8]: return '여름'
    elif month in [12, 1, 2]: return '겨울'
    else: return '봄가을'

elec_df['계절'] = elec_df['월'].apply(get_season)

# 계절별 평균 사용량 계산
seasonal_usage = elec_df.groupby(['시군구_병합_수정', '계절'])['가구당 평균 전력 사용량(kWh)'].mean().unstack(fill_value=0)

# 계절성 지수 계산: (여름+겨울) / (봄가을)
# 봄/가을 사용량이 0인 경우를 대비하여 0으로 처리
seasonal_usage['전력소비_계절성_지수'] = (seasonal_usage['여름'] + seasonal_usage['겨울']) / seasonal_usage['봄가을']
seasonal_usage.replace([np.inf, -np.inf], 0, inplace=True) # 0으로 나누는 경우 방지
seasonal_usage.fillna(0, inplace=True)

# --- 10-4. 최종 데이터 병합 ---
print("계산된 모든 전력 지표를 병합합니다...")
elec_final = pd.merge(elec_agg, seasonal_usage[['전력소비_계절성_지수']], on='시군구_병합_수정', how='left')
elec_final.rename(columns={'시군구_병합_수정': '시군구_병합'}, inplace=True)

print("전력 데이터 집계 완료.")
print("\n최종 생성된 전력 데이터(elec_final) 정보:")
elec_final.info()
print("\n최종 전력 데이터 샘플:")
display(elec_final.head())

시군구별 전력 사용량 관련 지표 계산을 시작합니다...
데이터 타입 변환 및 월 정보 추출 완료.
기본 지표(평균 사용량, 변동성, 효율)를 집계합니다...
전력소비 계절성 지수를 계산합니다...
계산된 모든 전력 지표를 병합합니다...
전력 데이터 집계 완료.

최종 생성된 전력 데이터(elec_final) 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232 entries, 0 to 231
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   시군구_병합        232 non-null    object 
 1   가구당_평균_전력사용량  232 non-null    float64
 2   전력소비_변동성_지수   232 non-null    float64
 3   전력_에너지_효율     232 non-null    float64
 4   전력소비_계절성_지수   232 non-null    float64
dtypes: float64(4), object(1)
memory usage: 9.2+ KB

최종 전력 데이터 샘플:


,시군구_병합,가구당_평균_전력사용량,전력소비_변동성_지수,전력_에너지_효율,전력소비_계절성_지수
0,강원특별자치도 강릉시,207.916667,0.124400,137.585972,2.290628
1,강원특별자치도 고성군,176.500000,0.133410,155.574599,2.274470
2,강원특별자치도 동해시,214.083333,0.123609,132.162709,2.260365
3,강원특별자치도 삼척시,179.666667,0.116734,144.802876,2.227451
4,강원특별자치도 속초시,228.166667,0.110384,130.537619,2.271451


In [30]:
# 11. 복지시설 관련 지표 계산

print("복지시설 관련 데이터를 처리하고 지표를 계산합니다...")

try:
    # --- 11-1. 데이터 로드 및 전처리 ---
    # 파일의 3번째 줄을 헤더로 사용하여 필요한 컬럼들을 직접 선택
    welfare_df = pd.read_csv(data_path + '사회복지시설수.csv', encoding='utf-8', header=2)
    # 컬럼 이름이 중복되므로 위치 기반으로 필요한 데이터 선택
    # [시도, 시군구, 인구수, 총시설수, 총생활인원, 아동시설수, 아동생활인원, 노인시설수, 노인생활인원, 장애인시설수, 장애인생활인원]
    cols_to_use = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    welfare_df = welfare_df.iloc[:, cols_to_use]
    welfare_df.columns = [
        '시도', '시군구', '인구수', '총_시설수', '총_생활인원수',
        '아동_시설수', '아동_생활인원수', '노인_시설수', '노인_생활인원수',
        '장애인_시설수', '장애인_생활인원수'
    ]

    # 숫자형으로 변환
    for col in welfare_df.columns[2:]:
        welfare_df[col] = pd.to_numeric(welfare_df[col], errors='coerce')
    
    # 불필요한 행 제거 및 지역명 통합
    welfare_df.dropna(inplace=True)
    welfare_df = welfare_df[welfare_df['시도'] != '총계']
    welfare_df = welfare_df[(welfare_df['시군구'] != '소계') | (welfare_df['시도'] == '세종특별자치시')].reset_index(drop=True)
    welfare_df['시군구_병합'] = welfare_df.apply(lambda r: r['시도'] if r['시도'] == '세종특별자치시' else r['시도'] + ' ' + r['시군구'], axis=1)
    welfare_df['시군구_병합_수정'] = welfare_df['시군구_병합'].apply(standardize_region_name)
    welfare_df.drop(columns=['시도', '시군구', '시군구_병합'], inplace=True)
    print("데이터 로드 및 기본 전처리 완료.")

    # --- 11-2. 비율 및 정규화 지표 계산 ---
    print("비율 및 정규화 지표를 계산합니다...")
    # 인구 1만명당 지표
    welfare_df['인구1만명당_총_복지시설_수'] = (welfare_df['총_시설수'] / welfare_df['인구수']) * 10000
    welfare_df['인구1만명당_총_생활인원_수'] = (welfare_df['총_생활인원수'] / welfare_df['인구수']) * 10000
    
    # 시설 비중 (%)
    welfare_df['노인복지시설_비중'] = (welfare_df['노인_시설수'] / welfare_df['총_시설수']) * 100
    welfare_df['아동_장애인_복지시설_비중'] = ((welfare_df['아동_시설수'] + welfare_df['장애인_시설수']) / welfare_df['총_시설수']) * 100
    
    # 시설당 평균 생활인원
    welfare_df['시설당_평균_생활인원'] = welfare_df['총_생활인원수'] / welfare_df['총_시설수']
    welfare_df.fillna(0, inplace=True) # 0으로 나누는 경우 대비

    # --- 11-3. 사회적 취약도 지수 계산 ---
    print("사회적 취약도 지수를 계산합니다...")
    # 각 분야별 인구 1만명당 생활인원수 계산
    welfare_df['인구1만명당_노인_생활인원'] = (welfare_df['노인_생활인원수'] / welfare_df['인구수']) * 10000
    welfare_df['인구1만명당_아동_생활인원'] = (welfare_df['아동_생활인원수'] / welfare_df['인구수']) * 10000
    welfare_df['인구1만명당_장애인_생활인원'] = (welfare_df['장애인_생활인원수'] / welfare_df['인구수']) * 10000

    # 표준화 (Standardization)
    for col in ['인구1만명당_노인_생활인원', '인구1만명당_아동_생활인원', '인구1만명당_장애인_생활인원']:
        mean = welfare_df[col].mean()
        std = welfare_df[col].std()
        welfare_df[col + '_표준화'] = (welfare_df[col] - mean) / std if std != 0 else 0

    # 표준화 점수 합산하여 종합 지수 생성
    welfare_df['사회적_취약도_지수'] = welfare_df[[col + '_표준화' for col in ['인구1만명당_노인_생활인원', '인구1만명당_아동_생활인원', '인구1만명당_장애인_생활인원']]].sum(axis=1)

    # --- 11-4. 최종 데이터 정리 ---
    cols_to_keep = [
        '시군구_병합_수정', '인구1만명당_총_복지시설_수', '인구1만명당_총_생활인원_수',
        '노인복지시설_비중', '아동_장애인_복지시설_비중', '시설당_평균_생활인원', '사회적_취약도_지수'
    ]
    welfare_final = welfare_df[cols_to_keep].copy()
    welfare_final.rename(columns={'시군구_병합_수정': '시군구_병합'}, inplace=True)
    
    print("복지시설 데이터 처리 완료.")
    print("\n최종 생성된 복지시설 데이터(welfare_final) 정보:")
    welfare_final.info()
    print("\n최종 복지시설 데이터 샘플:")
    display(welfare_final.head())

except FileNotFoundError:
    print(f"오류: 파일을 찾을 수 없습니다. ({data_path}사회복지시설수.csv)")
except Exception as e:
    print(f"데이터 처리 중 오류 발생: {e}")

복지시설 관련 데이터를 처리하고 지표를 계산합니다...
데이터 로드 및 기본 전처리 완료.
비율 및 정규화 지표를 계산합니다...
사회적 취약도 지수를 계산합니다...
복지시설 데이터 처리 완료.

최종 생성된 복지시설 데이터(welfare_final) 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   시군구_병합           207 non-null    object 
 1   인구1만명당_총_복지시설_수  207 non-null    float64
 2   인구1만명당_총_생활인원_수  207 non-null    float64
 3   노인복지시설_비중        207 non-null    float64
 4   아동_장애인_복지시설_비중   207 non-null    float64
 5   시설당_평균_생활인원      207 non-null    float64
 6   사회적_취약도_지수       207 non-null    float64
dtypes: float64(6), object(1)
memory usage: 11.4+ KB

최종 복지시설 데이터 샘플:


,시군구_병합,인구1만명당_총_복지시설_수,인구1만명당_총_생활인원_수,노인복지시설_비중,아동_장애인_복지시설_비중,시설당_평균_생활인원,사회적_취약도_지수
0,서울특별시 종로구,6.226572,55.787567,92.929293,3.030303,8.959596,-0.190553
1,서울특별시 용산구,5.232071,22.072799,95.312500,3.125000,4.218750,-1.280030
2,서울특별시 성북구,6.128986,31.383093,92.335766,4.014599,5.120438,-1.747089
3,서울특별시 강북구,6.547506,34.663269,95.098039,3.431373,5.294118,-1.907459
4,서울특별시 노원구,7.172240,37.360160,96.560847,3.174603,5.208995,-1.515987


In [31]:
# 12. 최종 데이터 병합

from functools import reduce

print("모든 개별 데이터를 하나의 종합 데이터프레임으로 병합합니다...")

# 병합할 데이터프레임 리스트
data_frames = [house_final, store_final, elec_final, welfare_final]

# '시군구_병합'을 기준으로 모든 데이터프레임 병합
# reduce 함수를 사용하여 리스트에 있는 데이터프레임을 순차적으로 병합
final_df = reduce(lambda left, right: pd.merge(left, right, on='시군구_병합', how='outer'), data_frames)

# 병합 후 생성될 수 있는 결측치 확인
missing_values = final_df.isnull().sum()
print("\n병합 후 컬럼별 결측치 개수:")
print(missing_values[missing_values > 0])

# 최종 결과 확인
print("\n최종 통합 데이터(final_df) 정보:")
final_df.info()
print("\n최종 통합 데이터 샘플:")
display(final_df.head())



모든 개별 데이터를 하나의 종합 데이터프레임으로 병합합니다...

병합 후 컬럼별 결측치 개수:
월세_평균_표준화          18
월세_표준편차_표준화        18
평균_평당_가격           18
건물_노후도_평균          18
거래량                18
지역별_소득수준           18
주거비_부담_지수          18
주택_시장_활력_지수        18
신규_주택_공급_지수        18
다양성_지수_새넌          16
다양성_지수_HHI         16
상가_개수_평균           16
상가_변화율             16
업종별상가수_과학·기술       16
업종별상가수_교육          16
업종별상가수_보건의료        16
업종별상가수_부동산         16
업종별상가수_소매          16
업종별상가수_수리·개인       16
업종별상가수_숙박          16
업종별상가수_시설관리·임대     16
업종별상가수_예술·스포츠      16
업종별상가수_음식          16
업종_특화도             16
인구수                17
면적(㎢)              31
인구_대비_상가_수         16
면적_대비_상가_수         16
가구당_평균_전력사용량       14
전력소비_변동성_지수        14
전력_에너지_효율          14
전력소비_계절성_지수        14
인구1만명당_총_복지시설_수    39
인구1만명당_총_생활인원_수    39
노인복지시설_비중          39
아동_장애인_복지시설_비중     39
시설당_평균_생활인원        39
사회적_취약도_지수         39
dtype: int64

최종 통합 데이터(final_df) 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245


,시군구_병합,월세_평균_표준화,월세_표준편차_표준화,평균_평당_가격,건물_노후도_평균,거래량,지역별_소득수준,주거비_부담_지수,주택_시장_활력_지수,신규_주택_공급_지수,다양성_지수_새넌,다양성_지수_HHI,상가_개수_평균,상가_변화율,업종별상가수_과학·기술,업종별상가수_교육,업종별상가수_보건의료,업종별상가수_부동산,업종별상가수_소매,업종별상가수_수리·개인,업종별상가수_숙박,업종별상가수_시설관리·임대,업종별상가수_예술·스포츠,업종별상가수_음식,업종_특화도,인구수,면적(㎢),인구_대비_상가_수,면적_대비_상가_수,가구당_평균_전력사용량,전력소비_변동성_지수,전력_에너지_효율,전력소비_계절성_지수,인구1만명당_총_복지시설_수,인구1만명당_총_생활인원_수,노인복지시설_비중,아동_장애인_복지시설_비중,시설당_평균_생활인원,사회적_취약도_지수
0,강원특별자치도 강릉시,33.651822,19.986456,2.852454,20.943791,2295.0,1160203.0,0.000348,48163.916951,0.047747,4.412261,276.430534,14603.50,0.031213,3172.0,2020.0,304.0,1351.0,12950.0,6784.0,4893.0,2040.0,2401.0,22499.0,1.0,215603.0,1041.0,67.733288,14.028338,207.916667,0.124400,137.585972,2.290628,23.840114,152.734424,96.498054,2.723735,6.406615,-0.174928
1,강원특별자치도 고성군,37.536465,15.465865,3.181730,11.154762,84.0,117857.0,0.003822,1383.132628,0.089648,3.992554,435.338219,2287.00,0.072836,344.0,212.0,22.0,193.0,1951.0,578.0,1978.0,283.0,392.0,3195.0,1.0,27869.0,664.0,82.062507,3.444277,176.500000,0.133410,155.574599,2.274470,44.135060,141.375722,99.186992,0.813008,3.203252,-0.407451
2,강원특별자치도 동해시,21.369696,13.275026,1.811375,19.161987,926.0,406705.0,0.000631,13218.674171,0.052187,4.432885,284.248181,5303.50,0.029242,895.0,656.0,117.0,221.0,5706.0,2716.0,859.0,737.0,835.0,8472.0,1.0,91492.0,180.0,57.966817,29.463889,214.083333,0.123609,132.162709,2.260365,17.815765,148.537577,98.159509,1.226994,8.337423,-0.704407
3,강원특별자치도 삼척시,34.362456,26.415448,2.912690,22.138298,282.0,248577.0,0.001659,7731.156358,0.045171,4.210664,368.206013,3866.25,0.069641,601.0,296.0,46.0,167.0,3722.0,1582.0,1480.0,581.0,578.0,6412.0,2.0,65939.0,1188.0,58.633737,3.254419,179.666667,0.116734,144.802876,2.227451,41.098591,106.916999,99.261993,0.738007,2.601476,-1.279197
4,강원특별자치도 속초시,42.095656,20.636705,3.568184,18.571325,1381.0,496214.0,0.001018,29880.289212,0.053846,4.325667,319.685911,6360.25,0.024616,1170.0,562.0,155.0,691.0,6360.0,2495.0,1372.0,702.0,1065.0,10869.0,2.0,83674.0,106.0,76.012262,60.002358,228.166667,0.110384,130.537619,2.271451,15.895021,112.818797,97.744361,1.503759,7.097744,-1.426355


In [32]:
# 13. 최종 데이터프레임 컬럼명 정리

print("최종 데이터프레임의 컬럼명을 규칙에 따라 정리합니다.")

# 분야별 컬럼 정의
# 주거 관련 컬럼
housing_cols = [
    '월세_평균_표준화', '월세_표준편차_표준화', '평균_평당_가격', '건물_노후도_평균', 
    '거래량', '지역별_소득수준', '주거비_부담_지수', '주택_시장_활력_지수', '신규_주택_공급_지수'
]
# 상가 관련 컬럼
store_cols = [
    '다양성_지수_새넌', '다양성_지수_HHI', '상가_개수_평균', '상가_변화율', 
    '업종별상가수_과학·기술', '업종별상가수_교육', '업종별상가수_보건의료', 
    '업종별상가수_부동산', '업종별상가수_소매', '업종별상가수_수리·개인', 
    '업종별상가수_숙박', '업종별상가수_시설관리·임대', '업종별상가수_예술·스포츠', 
    '업종별상가수_음식', '업종_특화도', '인구_대비_상가_수', '면적_대비_상가_수'
]
# 전력 관련 컬럼
elec_cols = [
    '가구당_평균_전력사용량', '전력소비_변동성_지수', '전력_에너지_효율', '전력소비_계절성_지수'
]
# 복지 관련 컬럼
welfare_cols = [
    '인구1만명당_총_복지시설_수', '인구1만명당_총_생활인원_수', '노인복지시설_비중',
    '아동_장애인_복지시설_비중', '시설당_평균_생활인원', '사회적_취약도_지수'
]

# 새로운 컬럼명을 저장할 딕셔너리 생성
new_column_names = {}
for col in final_df.columns:
    if col in housing_cols:
        new_column_names[col] = '주거_' + col
    elif col in store_cols and not col.startswith('상가_'):
        new_column_names[col] = '상가_' + col
    elif col in elec_cols and not col.startswith('전력_'):
        new_column_names[col] = '전력_' + col
    elif col in welfare_cols:
        new_column_names[col] = '복지_' + col
    else:
        # 그 외 컬럼은 이름 변경 없이 그대로 사용
        new_column_names[col] = col

# 컬럼명 변경 적용
final_df.rename(columns=new_column_names, inplace=True)

print("컬럼명 변경 완료.")
print("\n변경된 데이터프레임 정보:")
final_df.info()
print("\n변경된 데이터프레임 샘플:")
display(final_df.head())

최종 데이터프레임의 컬럼명을 규칙에 따라 정리합니다.
컬럼명 변경 완료.

변경된 데이터프레임 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   시군구_병합              246 non-null    object 
 1   주거_월세_평균_표준화        228 non-null    float64
 2   주거_월세_표준편차_표준화      228 non-null    float64
 3   주거_평균_평당_가격         228 non-null    float64
 4   주거_건물_노후도_평균        228 non-null    float64
 5   주거_거래량              228 non-null    float64
 6   주거_지역별_소득수준         228 non-null    float64
 7   주거_주거비_부담_지수        228 non-null    float64
 8   주거_주택_시장_활력_지수      228 non-null    float64
 9   주거_신규_주택_공급_지수      228 non-null    float64
 10  상가_다양성_지수_새넌        230 non-null    float64
 11  상가_다양성_지수_HHI       230 non-null    float64
 12  상가_개수_평균            230 non-null    float64
 13  상가_변화율              230 non-null    float64
 14  상가_업종별상가수_과학·기술     230 non-null    float64
 15  

,시군구_병합,주거_월세_평균_표준화,주거_월세_표준편차_표준화,주거_평균_평당_가격,주거_건물_노후도_평균,주거_거래량,주거_지역별_소득수준,주거_주거비_부담_지수,주거_주택_시장_활력_지수,주거_신규_주택_공급_지수,상가_다양성_지수_새넌,상가_다양성_지수_HHI,상가_개수_평균,상가_변화율,상가_업종별상가수_과학·기술,상가_업종별상가수_교육,상가_업종별상가수_보건의료,상가_업종별상가수_부동산,상가_업종별상가수_소매,상가_업종별상가수_수리·개인,상가_업종별상가수_숙박,상가_업종별상가수_시설관리·임대,상가_업종별상가수_예술·스포츠,상가_업종별상가수_음식,상가_업종_특화도,인구수,면적(㎢),상가_인구_대비_상가_수,상가_면적_대비_상가_수,전력_가구당_평균_전력사용량,전력_전력소비_변동성_지수,전력_에너지_효율,전력_전력소비_계절성_지수,복지_인구1만명당_총_복지시설_수,복지_인구1만명당_총_생활인원_수,복지_노인복지시설_비중,복지_아동_장애인_복지시설_비중,복지_시설당_평균_생활인원,복지_사회적_취약도_지수
0,강원특별자치도 강릉시,33.651822,19.986456,2.852454,20.943791,2295.0,1160203.0,0.000348,48163.916951,0.047747,4.412261,276.430534,14603.50,0.031213,3172.0,2020.0,304.0,1351.0,12950.0,6784.0,4893.0,2040.0,2401.0,22499.0,1.0,215603.0,1041.0,67.733288,14.028338,207.916667,0.124400,137.585972,2.290628,23.840114,152.734424,96.498054,2.723735,6.406615,-0.174928
1,강원특별자치도 고성군,37.536465,15.465865,3.181730,11.154762,84.0,117857.0,0.003822,1383.132628,0.089648,3.992554,435.338219,2287.00,0.072836,344.0,212.0,22.0,193.0,1951.0,578.0,1978.0,283.0,392.0,3195.0,1.0,27869.0,664.0,82.062507,3.444277,176.500000,0.133410,155.574599,2.274470,44.135060,141.375722,99.186992,0.813008,3.203252,-0.407451
2,강원특별자치도 동해시,21.369696,13.275026,1.811375,19.161987,926.0,406705.0,0.000631,13218.674171,0.052187,4.432885,284.248181,5303.50,0.029242,895.0,656.0,117.0,221.0,5706.0,2716.0,859.0,737.0,835.0,8472.0,1.0,91492.0,180.0,57.966817,29.463889,214.083333,0.123609,132.162709,2.260365,17.815765,148.537577,98.159509,1.226994,8.337423,-0.704407
3,강원특별자치도 삼척시,34.362456,26.415448,2.912690,22.138298,282.0,248577.0,0.001659,7731.156358,0.045171,4.210664,368.206013,3866.25,0.069641,601.0,296.0,46.0,167.0,3722.0,1582.0,1480.0,581.0,578.0,6412.0,2.0,65939.0,1188.0,58.633737,3.254419,179.666667,0.116734,144.802876,2.227451,41.098591,106.916999,99.261993,0.738007,2.601476,-1.279197
4,강원특별자치도 속초시,42.095656,20.636705,3.568184,18.571325,1381.0,496214.0,0.001018,29880.289212,0.053846,4.325667,319.685911,6360.25,0.024616,1170.0,562.0,155.0,691.0,6360.0,2495.0,1372.0,702.0,1065.0,10869.0,2.0,83674.0,106.0,76.012262,60.002358,228.166667,0.110384,130.537619,2.271451,15.895021,112.818797,97.744361,1.503759,7.097744,-1.426355


In [34]:
#최종 데이터프레임을 CSV 파일로 저장 (선택 사항)
final_df.to_csv(data_path + '시군구별_주거_상가_전력_복지_종합데이터터.csv', index=False, encoding='utf-8')
print("\n'final_merged_data.csv' 파일로 저장 완료.")


'final_merged_data.csv' 파일로 저장 완료.


### **최종 데이터 지표 정의서**

#### **1. 공통 데이터**
| 변수명 | 정의 | 데이터 출처 |
| :--- | :--- | :--- |
| `시군구_병합` | 분석의 기준이 되는 행정구역(시도-시군구) 명칭 | 모든 데이터에서 통합 |
| `인구수` | 지역의 주민등록인구수 (등록외국인 포함) | `사회복지시설수.csv` |
| `면적(㎢)` | 지역의 행정구역 면적 (단위: ㎢) | `시군구별면적.csv` |

<br>

#### **2. 주거 관련 지표**
| 변수명 | 정의 | 산출 수식 |
| :--- | :--- | :--- |
| `주거_월세_평균_표준화` | 지역별 1인당 표준 주거면적(수도권 34.2, 광역시 35.6, 도 39.0 ㎡)을 기준으로 환산한 월세의 평균값 | `(월세금(만원) / 전용면적(㎡) * 1인당 표준 주거면적)`의 시군구별 평균 |
| `주거_월세_표준편차_표준화` | '주거\_월세\_평균\_표준화'의 시군구별 표준편차. 가격 안정성을 나타냄 | `(월세금(만원) / 전용면적(㎡) * 1인당 표준 주거면적)`의 시군구별 표준편차 |
| `주거_평균_평당_가격` | 평(3.3㎡)당 월세 가격의 시군구별 평균 | `(월세금(만원) / (전용면적(㎡) * 0.3025))`의 시군구별 평균 |
| `주거_건물_노후도_평균` | 건축년도(기준 2023년)를 기준으로 계산한 건물의 평균 노후도 | `(2023 - 건축년도)`의 시군구별 평균 |
| `주거_거래량` | 해당 지역의 연간 총 월세 거래 건수 | `월세 데이터`의 시군구별 row 개수 |
| `주거_지역별_소득수준` | 지역의 연간 종합소득금액의 합 (단위: 백만원) | `시군구별_종합소득세.csv` |
| `주거_주거비_부담_지수` | 월 평균 소득에서 표준화된 월세가 차지하는 비율(%) | `(주거_월세_평균_표준화 / (월평균소득(만원))) * 100` |
| `주거_주택_시장_활력_지수` | 거래량과 가격 변동성을 함께 고려한 시장 활성화 지표 | `주거_거래량 * (1 + 주거_월세_표준편차_표준화)` |
| `주거_신규_주택_공급_지수` | 건물 노후도의 역수로, 신축 건물이 많을수록 높은 값을 가짐 | `1 / 주거_건물_노후도_평균` |

<br>

#### **3. 상가 관련 지표**
| 변수명 | 정의 | 산출 수식 |
| :--- | :--- | :--- |
| `상가_다양성_지수_새넌` | 업종의 다양성과 분포의 균일성을 나타내는 지수 (높을수록 다양) | Shannon-Wiener Index: `-\sum(p_i * ln(p_i))` |
| `상가_다양성_지수_HHI` | 업종의 집중도를 나타내는 지수 (낮을수록 다양) | Herfindahl-Hirschman Index: `\sum(p_i * 100)^2` |
| `상가_개수_평균` | 4개 분기에 걸친 지역 내 평균 상가 수 | `연간 총 상가 수 / 4` |
| `상가_변화율` | 1분기 대비 4분기의 상가 수 변화율 | `(4분기 상가 수 - 1분기 상가 수) / 1분기 상가 수` |
| `상가_업종별상가수_*` | '음식', '소매' 등 주요 대분류 업종별 총 상가 수 | `상권업종대분류명` 기준 pivot table |
| `상가_업종_특화도` | 지역의 특정 업종 집중도를 나타내는 입지계수(LQ) 기반 지표 | `LQ > 1.25`를 만족하는 특화 업종의 개수 |
| `상가_인구_대비_상가_수` | 인구 1,000명당 평균 상가 수 | `(상가_개수_평균 / 인구수) * 1000` |
| `상가_면적_대비_상가_수` | 단위 면적(㎢)당 평균 상가 수 | `상가_개수_평균 / 면적(㎢)` |

<br>

#### **4. 전력 관련 지표**
| 변수명 | 정의 | 산출 수식 |
| :--- | :--- | :--- |
| `전력_가구당_평균_전력사용량` | 가구당 월평균 전력 사용량(kWh)의 연간 평균 | `가구당 평균 전력 사용량(kWh)`의 시군구별 연간 평균 |
| `전력_전력소비_변동성_지수` | 월별 전력 사용량의 변동계수. 값이 클수록 월별 편차가 큼 | `월별 사용량 표준편차 / 월별 사용량 평균` |
| `전력_에너지_효율` | 1kWh당 평균 전기요금 (단위: 원) | `가구당 평균 전기요금(원) / 가구당 평균 전력 사용량(kWh)` |
| `전력_전력소비_계절성_지수` | 봄/가을 대비 여름/겨울의 전력 소비 패턴을 나타내는 비율 | `(여름철+겨울철 평균 사용량) / (봄/가을철 평균 사용량)` |

<br>

#### **5. 복지 관련 지표**
| 변수명 | 정의 | 산출 수식 |
| :--- | :--- | :--- |
| `복지_인구1만명당_총_복지시설_수` | 인구 1만 명당 총 복지시설 수 | `(총 시설 수 / 인구수) * 10000` |
| `복지_인구1만명당_총_생활인원_수` | 인구 1만 명당 복지시설 총 생활인원 수 | `(총 생활인원 수 / 인구수) * 10000` |
| `복지_노인복지시설_비중` | 전체 복지시설 중 노인복지시설이 차지하는 비율(%) | `(노인복지시설 수 / 총 시설 수) * 100` |
| `복지_아동_장애인_복지시설_비중`| 전체 복지시설 중 아동 및 장애인 복지시설이 차지하는 비율(%) | `((아동시설 수 + 장애인시설 수) / 총 시설 수) * 100` |
| `복지_시설당_평균_생활인원` | 복지시설 1개소당 평균 생활인원 수 | `총 생활인원 수 / 총 시설 수` |
| `복지_사회적_취약도_지수` | 노인, 아동, 장애인 관련 지표를 종합한 지역의 사회적 취약성 점수 | `노인/아동/장애인별 인구 1만명당 생활인원`을 각각 표준화하여 합산 |
